In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import GPy, patsy, scipy, popmachine

In [ ]:
def transform(z, mean, std):
    return (z - mean) / std

In [ ]:
def reverseTransform(z, mean, std):
    return z*std + mean

In [ ]:
def plotPrediction(xpred, predict_func, n, **kwargs):
    noise_dict = {'output_index':xpred[:,-1:].astype(int)}
    mu, var = predict_func(xpred,Y_metadata=noise_dict)
    std = np.sqrt(var[:,0])
    mu = mu[:,0]

    plt.plot(reverseTransform(xpred[:,0],*norm['time']), reverseTransform(mu, *n), **kwargs)
    plt.fill_between(reverseTransform(xpred[:,0],*norm['time']), reverseTransform(mu-1.98*std, *n), reverseTransform(mu+1.98*std, *n), alpha=.2, **kwargs)

In [ ]:
machine = popmachine.Machine()

In [ ]:
machine.plates(names='True')

In [ ]:
ds = machine.search('cfu-od-1',include=['strain', 'measurment'])

In [ ]:
ds.data.head()

In [ ]:
ds.meta.head()

In [ ]:
melt = ds.melt()
del melt['number']
del melt['plate']

od = melt[melt.measurment=='od']
cfu = melt[melt.measurment=='cfu']

del od['measurment']

cfu['cfu'] = cfu.od
del cfu['od']
del cfu['measurment']

In [ ]:
od.head()

In [ ]:
cfu.tail()

In [ ]:
merge = pd.merge(od, cfu, 'outer')

merge['logCFU'] = np.log10(merge['cfu'])
merge['logOD'] = np.log2(merge.od)
merge.time = merge.time.astype(float)
merge['normtime'] = (merge.time-merge.time.mean())/merge.time.std()

merge.tail()

In [ ]:
plt.figure(figsize=(6,4))

plt.scatter(merge.time, merge.logOD,c='C0')
plt.ylabel("$log_2($OD$)$",fontsize=15,color='C0')
plt.yticks(color='C0')
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.twinx()
plt.scatter(merge.time, merge.logCFU,c='C1')
plt.ylabel("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.yticks(color='C1')

plt.savefig("figures/cfu-vs-od.pdf",bbox_inches='tight')

In [ ]:
plt.scatter(merge.logOD, merge.logCFU)
plt.ylabel("$log_{10}($CFU$)$",fontsize=15)
plt.xlabel("$log_2($OD$)$",fontsize=15)

plt.savefig("figures/cfu-vs-od-corr.pdf",bbox_inches='tight')

In [ ]:
select = ~merge.logCFU.isnull()
scipy.stats.pearsonr(merge.logOD[select], merge.logCFU[select])

In [ ]:
x1 = merge.normtime.values[:,None]
y1 = merge.logOD.values[:,None]

x2 = merge.normtime.values[~merge.logCFU.isnull(),None]
y2 = merge.logCFU.values[~merge.logCFU.isnull(),None]
# x2 = merge.normtime.values[:,None]
# y2 = merge.logCFU.values[:,None]

norm = {'y1': (y1.mean(), y1.std()),\
        'y2': (y2.mean(), y2.std()),\
        'time': (merge.time.mean(), merge.time.std())}

y1 = (y1-y1.mean())/y1.std()
y2 = (y2-y2.mean())/y2.std()

x1.shape, y1.shape, x2.shape, y2.shape,

In [ ]:
plt.scatter(x1,y1)
plt.scatter(x2,y2)

In [ ]:
lcm = GPy.util.multioutput.LCM(1,1,[GPy.kern.RBF(1) for i in range(2)])

In [ ]:
lcm

In [ ]:
K = GPy.kern.Matern32(1)
# K = GPy.kern.Matern52(1)
kern = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=K,W_rank=2)

# kern = GPy.util.multioutput.LCM(input_dim=1,num_outputs=2,kernels_list=[GPy.kern.Matern32(1) for i in range(2)])

m = GPy.models.GPCoregionalizedRegression([x1,x2],[y1,y2],kernel=kern)
m['.*Mat.*.var'].constrain_fixed(1.)

# icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=GPy.kern.RBF(1))
# m = GPy.models.GPCoregionalizedRegression([x1,x2],[y1,y2],kernel=icm)
# m.kern.rbf.variance.constrain_fixed(1.)

m.randomize()
m.optimize()
m

In [ ]:
# kern.ICM0.B.W

In [ ]:
# kern.ICM0.B.kappa

In [ ]:
# kern.ICM1.B.W

In [ ]:
# kern.ICM1.B.kappa

In [ ]:
# kern.B.W

In [ ]:
# icm.B.kappa

In [ ]:
kern.B.B[1,0]/np.sqrt(kern.B.B[0,0])/np.sqrt(kern.B.B[1,1])

In [ ]:
plt.figure(figsize=(8,4))

xpred = np.hstack([np.linspace(x1.min(), x1.max())[:,None], [[0]]*50])

plt.subplot(121)
plt.title("$log_2($OD$)$",fontsize=15,color='C0')
plt.xlabel("time (h)", fontsize=15)
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1, *norm['time']), reverseTransform(y1[:,0],*norm['y1']),c='C0')

plt.subplot(122)
plt.title("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.xlabel("time (h)", fontsize=15)
xpred[:,1] = 1
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2, *norm['time']), reverseTransform(y2[:,0],*norm['y2']),c='C1')

plt.savefig("figures/cfu-vs-od_gp.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8,4))

xpred = np.hstack([np.linspace(x1.min()*2.5, x1.max()*2.5)[:,None], [[0]]*50])

plt.subplot(121)
plt.title("$log_2($OD$)$",fontsize=15,color='C0')
plt.xlabel("time (h)", fontsize=15)
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1, *norm['time']), reverseTransform(y1[:,0],*norm['y1']),c='C0')

plt.subplot(122)
plt.title("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.xlabel("time (h)", fontsize=15)
xpred[:,1] = 1
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2, *norm['time']), reverseTransform(y2[:,0],*norm['y2']),c='C1')

plt.savefig("figures/cfu-vs-od-long_gp.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8,4))

xpred = np.hstack([np.linspace(x1.min()*2.5, x1.max()*2.5)[:,None], [[0]]*50])

plt.subplot(121)
plt.title("$log_2($OD$)$",fontsize=15,color='C0')
plt.xlabel("time (h)", fontsize=15)
plotPrediction(xpred, m.predict, norm['y1'])
plt.scatter(reverseTransform(x1, *norm['time']), reverseTransform(y1[:,0],*norm['y1']),c='C0')

plt.subplot(122)
plt.title("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.xlabel("time (h)", fontsize=15)
xpred[:,1] = 1
plotPrediction(xpred, m.predict, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2, *norm['time']), reverseTransform(y2[:,0],*norm['y2']),c='C1')

plt.savefig("figures/cfu-vs-od-obs-long_gp.pdf",bbox_inches='tight')

In [ ]:
xpred = np.hstack([np.tile(np.linspace(x1.min(), x1.max()),2)[:,None], [[0]]*100])
xpred[50:,1] = 1
noise_dict = {'output_index':xpred[:,1:].astype(int)}
mu, cov = m.predict_noiseless(xpred,Y_metadata=noise_dict,full_cov=True)
mu = mu[:,0]

# y = Bx + c
B = np.zeros((100,100))
B[range(100),range(100)] = [norm['y1'][1]]*50 + [norm['y2'][1]]*50

c = np.zeros(100)
c[:50] = norm['y1'][0]
c[50:] = norm['y2'][0]

mu = np.dot(B, mu) + c
cov = np.dot(B, np.dot(cov, B.T))


B = np.eye(100)
# convert cfu to log2
# B[range(50,100),range(50,100)] = 1./np.log10(2)
# ..or convert OD to log10
B[range(50),range(50)] = 1./np.log2(10)

mu = np.dot(B, mu)
cov = np.dot(B, np.dot(cov, B.T))

# compute diff
B = np.zeros((50,100))
B[range(50),range(50)] = -1.
B[range(50),range(50,100)] = 1.

mu = np.dot(B, mu)
cov = np.dot(B, np.dot(cov, B.T))
std = np.sqrt(np.diag(cov))

plt.figure(figsize=(4,4))

plt.plot(reverseTransform(xpred[:50,0],*norm['time']), mu, color='C4')
plt.fill_between(reverseTransform(xpred[:50,0],*norm['time']), mu-1.98*std, mu+1.98*std,alpha=.4, color='C4')

plt.title("$log_{10}($CFU/OD$)$",fontsize=15)
plt.xlabel("time (h)", fontsize=15)

plt.savefig("figures/cfu-vs-od_gp-ratio.pdf",bbox_inches='tight')

In [ ]:
K = GPy.kern.Matern32(1)

m1 = GPy.models.GPRegression(x1,y1,kernel=K.copy())
m1.optimize()
m2 = GPy.models.GPRegression(x2,y2,kernel=K.copy())
m2.optimize()

In [ ]:
plt.figure(figsize=(8,4))

xpred = np.hstack([np.linspace(x1.min(), x1.max())[:,None], [[0]]*50])

plt.subplot(121)
plt.title("$log_2($OD$)$",fontsize=15,color='C0')
plt.xlabel("time (h)", fontsize=15)
plotPrediction(xpred, m1.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1, *norm['time']), reverseTransform(y1[:,0],*norm['y1']),c='C0')

plt.subplot(122)
plt.title("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.xlabel("time (h)", fontsize=15)
xpred[:,1] = 1
plotPrediction(xpred, m2.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2, *norm['time']), reverseTransform(y2[:,0],*norm['y2']),c='C1')
plt.ylim(6.1, 9.5)

plt.savefig("figures/cfu-vs-od_gp-null.pdf")

In [ ]:
plt.figure(figsize=(8,4))

xpred = np.hstack([np.linspace(x1.min()*2.5, x1.max()*2.5)[:,None], [[0]]*50])

plt.subplot(121)
plt.title("$log_2($OD$)$",fontsize=15,color='C0')
plt.xlabel("time (h)", fontsize=15)
plotPrediction(xpred, m1.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1, *norm['time']), reverseTransform(y1[:,0],*norm['y1']),c='C0')

plt.subplot(122)
plt.title("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.xlabel("time (h)", fontsize=15)
xpred[:,1] = 1
plotPrediction(xpred, m2.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2, *norm['time']), reverseTransform(y2[:,0],*norm['y2']),c='C1')
plt.ylim(6.1, 9.5)

plt.savefig("figures/cfu-vs-od-long_gp-null.pdf",bbox_inches='tight')

# Add a second dataset 

In [ ]:
ds2 = machine.search('ura3-replicate', Strain = 'ura3', Bio='A', **{'mM_PQ':[0.0]})
print ds2

In [ ]:
melt2 = ds2.melt()
melt2['logOD'] = np.log2(melt2.od)
melt2['normtime'] = (melt2.time-norm['time'][0])/norm['time'][1]

# melt2 = melt2.iloc[::10,:]

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(121)
plt.scatter(merge.time, merge.logOD,c='C0')
plt.ylabel("$log_2($OD$)$",fontsize=15,color='C0')
plt.yticks(color='C0')
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.scatter(melt2.time, melt2.logOD,c='C2')

plt.twinx()
plt.scatter(merge.time, merge.logCFU,c='C1')
plt.ylabel("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.yticks(color='C1')

# plt.subplot(122)
# plt.scatter(melt2.time, melt2.logOD,c='C2')

plt.tight_layout()

# plt.savefig("figures/cfu-vs-od.pdf",bbox_inches='tight')

In [ ]:
y11 = merge.od.values
y12 = melt2.od.values

y12 = 2.499*y12 - 0.2155

y11 = np.log2(y11)
y12 = np.log2(y12)

x11 = merge.time.values
x12 = melt2.time.values + 20

x1 = np.zeros((x11.shape[0] + x12.shape[0], 2))
x1[:x11.shape[0],0] = x11
x1[x11.shape[0]:,0] = x12
x1[x11.shape[0]:,1] = 1

y1 = np.concatenate((y11, y12))[:,None]

x2 = np.zeros((sum(~merge.logCFU.isnull()), 2))
x2[:,0] = merge.time.values[~merge.logCFU.isnull()]

# x2 = merge.time.values[~merge.logCFU.isnull(),None]
y2 = merge.logCFU.values[~merge.logCFU.isnull(),None]

In [ ]:
plt.scatter(x11, y11)
plt.scatter(x12, y12, c="C2")

plt.twinx()
plt.scatter(x2[:,0],y2,c='C1')

In [ ]:
norm = {'y1': (y1.mean(), y1.std()),\
        'y2': (y2.mean(), y2.std()),\
        'time': (x1[:,0].mean(), x1[:,0].std())}

y1 = (y1-y1.mean())/y1.std()
y2 = (y2-y2.mean())/y2.std()
x2[:,0] = (x2[:,0]-x1[:,0].mean())/x1[:,0].std()
x1[:,0] = (x1[:,0]-x1[:,0].mean())/x1[:,0].std()

In [ ]:
plt.scatter(x1[:,0], y1)
plt.scatter(x2[:,0], y2)

In [ ]:
x1.shape, y1.shape, x2.shape, y2.shape

In [ ]:
# K = GPy.kern.Hierarchical([GPy.kern.Matern32(1), GPy.kern.Matern32(1)])
K = GPy.kern.Matern32(2)
# K = GPy.kern.RBF(2)

kern = GPy.util.multioutput.ICM(input_dim=2,num_outputs=2,kernel=K,W_rank=2)

m = GPy.models.GPCoregionalizedRegression([x1,x2],[y1,y2],kernel=kern)
# m['.*Mat.*.var'].constrain_fixed(1.)
# m['.*rbf.*.var'].constrain_fixed(1.)

m.randomize()
m.optimize()
m

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(x11, y11, color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50, [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'],color='C2')
plt.scatter(x12, y12, color='C2',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), reverseTransform(y2,*norm['y2']), color='C1')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50, [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C4')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.savefig("figures/cfu-vs-od_gp-extradata.pdf",bbox_inches='tight')

In [ ]:
ds2 = machine.search('ura3-replicate', Strain = 'ura3', Bio='A', **{'mM_PQ':[0.0,1.333]})
print ds2

## And second condition

In [ ]:
melt2 = ds2.melt()
melt2['logOD'] = np.log2(melt2.od)
melt2['normtime'] = (melt2.time-norm['time'][0])/norm['time'][1]

# melt2 = melt2.iloc[::5,:]

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(121)
plt.scatter(merge.time, merge.logOD,c='C0')
plt.ylabel("$log_2($OD$)$",fontsize=15,color='C0')
plt.yticks(color='C0')
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.scatter(melt2.time, melt2.logOD,c='C2')

plt.twinx()
plt.scatter(merge.time, merge.logCFU,c='C1')
plt.ylabel("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.yticks(color='C1')

# plt.subplot(122)
# plt.scatter(melt2.time, melt2.logOD,c='C2')

plt.tight_layout()

# plt.savefig("figures/cfu-vs-od.pdf",bbox_inches='tight')

In [ ]:
y11 = merge.od.values
y12 = melt2.od.values

y12 = 2.499*y12 - 0.2155

y11 = np.log2(y11)
y12 = np.log2(y12)

x11 = merge.time.values
x12 = melt2.time.values + 20

x1 = np.zeros((x11.shape[0] + x12.shape[0], 3))
x1[:x11.shape[0],0] = x11
x1[x11.shape[0]:,0] = x12

select = np.where(np.abs(melt2.mM_PQ.astype(float))>1e-9)[0]
x1[x11.shape[0]+select,1] = 1
x1[x11.shape[0]:,2] = 1

y1 = np.concatenate((y11, y12))[:,None]

x2 = np.zeros((sum(~merge.logCFU.isnull()), 3))
x2[:,0] = merge.time.values[~merge.logCFU.isnull()]

# x2 = merge.time.values[~merge.logCFU.isnull(),None]
y2 = merge.logCFU.values[~merge.logCFU.isnull(),None]

In [ ]:
plt.scatter(x11, y11)
plt.scatter(x12, y12, c="C2")

plt.twinx()
plt.scatter(x2[:,0],y2,c='C1')

In [ ]:
norm = {'y1': (y1.mean(), y1.std()),\
        'y2': (y2.mean(), y2.std()),\
        'time': (x1[:,0].mean(), x1[:,0].std())}

y1 = (y1-y1.mean())/y1.std()
y2 = (y2-y2.mean())/y2.std()
x2[:,0] = (x2[:,0]-x1[:,0].mean())/x1[:,0].std()
x1[:,0] = (x1[:,0]-x1[:,0].mean())/x1[:,0].std()

In [ ]:
plt.scatter(x1[:,0], y1)
plt.scatter(x2[:,0], y2)

In [ ]:
x1.shape, y1.shape, x2.shape, y2.shape

In [ ]:
# K = GPy.kern.Hierarchical([GPy.kern.Matern32(1), GPy.kern.Matern32(1)])
K = GPy.kern.Matern32(3)
# K = GPy.kern.RBF(2)

kern = GPy.util.multioutput.ICM(input_dim=3,num_outputs=2,kernel=K,W_rank=2)

m = GPy.models.GPCoregionalizedRegression([x1,x2],[y1,y2],kernel=kern)
# m['.*Mat.*.var'].constrain_fixed(1.)
# m['.*rbf.*.var'].constrain_fixed(1.)

m.randomize()
m.optimize()
m

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[0]]*50, [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(x11, y11, color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[1]]*50, [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'],color='C2')
plt.scatter(x12[melt2.mM_PQ.astype(float)==0.0], y12[melt2.mM_PQ.astype(float)==0.0], color='C2',s=10,alpha=.6)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50, [[1]]*50, [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'],color='C9')
plt.scatter(x12[melt2.mM_PQ.astype(float)!=0.0], y12[melt2.mM_PQ.astype(float)!=0.0], color='C9',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[0]]*50, [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), reverseTransform(y2,*norm['y2']), color='C1')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50, [[1]]*50, [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C4')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50, [[1]]*50, [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C9')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.savefig("figures/cfu-vs-od_gp-extradata-extracondition.pdf",bbox_inches='tight')

## Treat as third function 

### no transform 

In [ ]:
ds2 = machine.search('ura3-replicate', Strain = 'ura3', Bio='A', **{'mM_PQ':[0.0]})

melt2 = ds2.melt()
melt2['logOD'] = np.log2(melt2.od)
melt2['normtime'] = (melt2.time-melt2.time.mean())/melt2.time.std()

In [ ]:
plt.scatter(merge.time, merge.logOD)
plt.scatter(melt2.time+20, melt2.logOD, c="C2")

plt.ylabel("$log_2($OD$)$",fontsize=15,color='C0')
plt.yticks(color='C0')
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)


plt.twinx()
plt.scatter(merge.time,merge.logCFU,c='C1')

plt.ylabel("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.yticks(color='C1')

plt.savefig("figures/cfu-vs-od-flask&bioscreen.pdf",bbox_inches='tight')

In [ ]:
plt.scatter(merge.time, merge.logOD)
plt.scatter(melt2.time+20, np.log2(melt2.od*2.499 - 0.2155), c="C2")

plt.ylabel("$log_2($OD$)$",fontsize=15,color='C0')
plt.yticks(color='C0')
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)


plt.twinx()
plt.scatter(merge.time,merge.logCFU,c='C1')

plt.ylabel("$log_{10}($CFU$)$",fontsize=15,color='C1')
plt.yticks(color='C1')

plt.savefig("figures/cfu-vs-od-flask&bioscreen-transformed.pdf",bbox_inches='tight')

In [ ]:
x1 = merge.time.values[:,None]
y1 = merge.logOD.values[:,None]

x2 = merge.time.values[~merge.logCFU.isnull(),None]
y2 = merge.logCFU.values[~merge.logCFU.isnull(),None]
# x2 = merge.normtime.values[:,None]
# y2 = merge.logCFU.values[:,None]

x3 = (melt2.time.values+20)[:,None]
y3 = melt2.od.values
# y3 = 2.499*y3 - 0.2155
y3 = np.log2(y3)[:,None]

norm = {'y1': (y1.mean(), y1.std()),\
        'y2': (y2.mean(), y2.std()),\
        'y3': (y3.mean(), y3.std()),\
        'time': (merge.time.mean(), merge.time.std())}

y1 = (y1-y1.mean())/y1.std()
y2 = (y2-y2.mean())/y2.std()
y3 = (y3-y3.mean())/y3.std()

x1 = transform(x1, *norm['time'])
x2 = transform(x2, *norm['time'])
x3 = transform(x3, *norm['time'])

x1.shape, y1.shape, x2.shape, y2.shape, x3.shape, y3.shape

In [ ]:
plt.scatter(x1, y1)
plt.scatter(x3, y3, c="C2")

plt.twinx()
plt.scatter(x2[:,0],y2,c='C1')

In [ ]:
K = GPy.kern.Matern32(1)
kern = GPy.util.multioutput.ICM(input_dim=1,num_outputs=3,kernel=K,W_rank=2)

m = GPy.models.GPCoregionalizedRegression([x1,x2,x3],[y1,y2,y3],kernel=kern)
m['.*Mat.*.var'].constrain_fixed(1.)
# m['.*rbf.*.var'].constrain_fixed(1.)

m.randomize()
m.optimize()
m

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1))
plt.scatter(reverseTransform(x1[:,0],*norm['time']), y1, color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[2]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1),color='C2')
plt.scatter(reverseTransform(x3[:,0],*norm['time']), y3, color='C2',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1),color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), y2, color='C1')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

# plt.savefig("figures/cfu-vs-od_gp-extradata.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1[:,0],*norm['time']), reverseTransform(y1,*norm['y1']), color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[2]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y3'],color='C2')
plt.scatter(reverseTransform(x3[:,0],*norm['time']), reverseTransform(y3,*norm['y3']), color='C2',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), reverseTransform(y2,*norm['y2']), color='C1')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.savefig("figures/coregionalize-CFU-ODflask-ODbioscreen(no transform)-gp.pdf",bbox_inches='tight')

### with transform 

In [ ]:
x1 = merge.time.values[:,None]
y1 = merge.logOD.values[:,None]

x2 = merge.time.values[~merge.logCFU.isnull(),None]
y2 = merge.logCFU.values[~merge.logCFU.isnull(),None]
# x2 = merge.normtime.values[:,None]
# y2 = merge.logCFU.values[:,None]

x3 = (melt2.time.values+20)[:,None]
y3 = melt2.od.values
y3 = 2.499*y3 - 0.2155
y3 = np.log2(y3)[:,None]

norm = {'y1': (y1.mean(), y1.std()),\
        'y2': (y2.mean(), y2.std()),\
        'y3': (y3.mean(), y3.std()),\
        'time': (merge.time.mean(), merge.time.std())}

y1 = (y1-y1.mean())/y1.std()
y2 = (y2-y2.mean())/y2.std()
y3 = (y3-y3.mean())/y3.std()

x1 = transform(x1, *norm['time'])
x2 = transform(x2, *norm['time'])
x3 = transform(x3, *norm['time'])

x1.shape, y1.shape, x2.shape, y2.shape, x3.shape, y3.shape

In [ ]:
plt.scatter(x1, y1)
plt.scatter(x3, y3, c="C2")

plt.twinx()
plt.scatter(x2[:,0],y2,c='C1')

In [ ]:
K = GPy.kern.Matern32(1)
kern = GPy.util.multioutput.ICM(input_dim=1,num_outputs=3,kernel=K,W_rank=2)

m = GPy.models.GPCoregionalizedRegression([x1,x2,x3],[y1,y2,y3],kernel=kern)
m['.*Mat.*.var'].constrain_fixed(1.)
# m['.*rbf.*.var'].constrain_fixed(1.)

m.randomize()
m.optimize()
m

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1))
plt.scatter(reverseTransform(x1[:,0],*norm['time']), y1, color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[2]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1),color='C2')
plt.scatter(reverseTransform(x3[:,0],*norm['time']), y3, color='C2',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, (0,1),color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), y2, color='C1')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

# plt.savefig("figures/cfu-vs-od_gp-extradata.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[0]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y1'])
plt.scatter(reverseTransform(x1[:,0],*norm['time']), reverseTransform(y1,*norm['y1']), color='C0')

xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[2]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y3'],color='C2')
plt.scatter(reverseTransform(x3[:,0],*norm['time']), reverseTransform(y3,*norm['y3']), color='C2',s=10,alpha=.6)

plt.title("$log_2($OD$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.subplot(122)
xpred = np.hstack([np.linspace(1.5*x1[:,0].min(), x1[:,0].max()*2)[:,None], [[1]]*50])
plotPrediction(xpred, m.predict_noiseless, norm['y2'],color='C1')
plt.scatter(reverseTransform(x2[:,0],*norm['time']), reverseTransform(y2,*norm['y2']), color='C1')

plt.title("$log_{10}($CFU$)$",fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("time (h)", fontsize=15)

plt.savefig("figures/coregionalize-CFU-ODflask-ODbioscreen(with transform)-gp.pdf",bbox_inches='tight')

### Add second condition